In [1]:
import sqlite3

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn


def create_table(conn):
    """ create a table from the create_table_sql statement
    """
    create_table_sql = """ CREATE TABLE IF NOT EXISTS students (
                                        matricula TEXT PRIMARY KEY,
                                        nombre TEXT NOT NULL,
                                        edad INTEGER,
                                        carrera TEXT
                                    ); """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

def insert_student(conn, student):
    """
    Insert a new student into the students table
    """
    sql = ''' INSERT INTO students(matricula,nombre,edad,carrera)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, student)
    conn.commit()
    return cur.lastrowid

def display_students(conn):
    """
    Query all rows in the students table
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM students")

    rows = cur.fetchall()

    print("--- Student Data ---")
    if not rows:
        print("No student data available.")
        return

    for row in rows:
        print(f"Matricula: {row[0]}")
        print(f"  Nombre: {row[1]}")
        print(f"  Edad: {row[2]}")
        print(f"  Carrera: {row[3]}")
        print("-" * 20)

def modify_student(conn, matricula, field, new_value):
    """
    Update a specific field for a student by matricula
    """
    sql = f''' UPDATE students
               SET {field} = ?
               WHERE matricula = ?'''
    cur = conn.cursor()
    try:
        cur.execute(sql, (new_value, matricula))
        conn.commit()
        if cur.rowcount > 0:
             print(f"Successfully updated {field} for student {matricula}")
        else:
             print(f"Student with matricula {matricula} not found or field '{field}' does not exist")
    except sqlite3.Error as e:
        print(f"Error updating student data: {e}")


def delete_student(conn, matricula):
    """
    Delete a student by matricula
    """
    sql = 'DELETE FROM students WHERE matricula=?'
    cur = conn.cursor()
    cur.execute(sql, (matricula,))
    conn.commit()
    if cur.rowcount > 0:
        print(f"Successfully deleted student with matricula {matricula}")
    else:
        print(f"Student with matricula {matricula} not found")


database = "students.db"


conn = create_connection(database)

if conn is not None:
    create_table(conn)

    student_data = ("231690117-8", "Edwin Montoya Aguilar", 17, "CDIA")
    try:
        insert_student(conn, student_data)
    except sqlite3.IntegrityError:
        print(f"Student with matricula {student_data[0]} already exists.")

    display_students(conn)

    modify_student(conn, "231690117-8", "nombre", "Josua Pastillas Garcia")

    display_students(conn)

    delete_student(conn, "231690117-8")

    display_students(conn)

    conn.close()
else:
    print("Error! cannot create the database connection.")

Student with matricula 231690117-8 already exists.
--- Student Data ---
Matricula: 231690117-8
  Nombre: Edwin Montoya Aguilar
  Edad: 17
  Carrera: CDIA
--------------------
Successfully updated nombre for student 231690117-8
--- Student Data ---
Matricula: 231690117-8
  Nombre: Josua Pastillas Garcia
  Edad: 17
  Carrera: CDIA
--------------------
Successfully deleted student with matricula 231690117-8
--- Student Data ---
No student data available.
